<a href="https://colab.research.google.com/github/hws2002/Deep_Learning_with_Keras/blob/main/Chapter11_4_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def self_attention(input_sequence):
  output = np.zeros(shape = input_sequence.shape)
  for i, pivot_vector in enumerate(input_sequence):
    scores = np.zeros(shape = (len(input_sequence),))
    for j, vector in enumerate(input_sequence):
      scores[j] = np.dot(pivot_vector, vector.T)
    scores /= np.sqrt(input_sequence.shape[1])
    scores = softmax(scores)
    new_pivot_representation = np.zeros(shape = pivot_vector.shape)

    for j, vector in enumerate(input_sequence):
        new_pivot_representation += vector * scores[j]
    output[i] = new_pivot_representation
  return output

In [ ]:
num_heads = 4
embed_dim = 256

mha_layer = MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
outputs = mha_layer(inputs, inputs, inputs)

# Layer 층을 상속하여 구현한 트랜스포머 인코디

In [1]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  35.0M      0  0:00:02  0:00:02 --:--:-- 35.0M


In [2]:
import os

base_path = os.path.join("aclImdb")

base_dir = 'aclImdb'
train_dir = 'train/'
test_dir = 'test/'

train_dataset = []
val_dataset = [] # 20%만 떼어가자

train_pos_path = os.path.join(base_path, train_dir, 'pos')
train_neg_path = os.path.join(base_path, train_dir, 'neg')

train_pos_listdir = os.listdir(train_pos_path)
num_val_pos_data = int(0.2 * len(train_pos_listdir))
train_neg_listdir = os.listdir(train_neg_path)
num_val_neg_data = int(0.2 * len(train_neg_listdir))

for filename in train_pos_listdir[:-num_val_pos_data]:
  with open(os.path.join(train_pos_path,filename),'r') as f:
    train_dataset.append(f.read())
for filename in train_pos_listdir[-num_val_pos_data:]:
  with open(os.path.join(train_pos_path,filename),'r') as f:
    val_dataset.append(f.read())

num_train_pos_data = len(train_dataset)

for filename in train_neg_listdir[:-num_val_neg_data]:
  with open(os.path.join(train_neg_path,filename),'r') as f:
    train_dataset.append(f.read())

for filename in train_neg_listdir[-num_val_neg_data:]:
  with open(os.path.join(train_neg_path,filename),'r') as f:
    val_dataset.append(f.read())
num_train_neg_data = len(train_dataset) - num_train_pos_data

print(len(train_dataset))
print(len(val_dataset))

20000
5000


In [3]:
import pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'

for category in ("neg","pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int( 0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname ,
                val_dir / category / fname)

In [4]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size = batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim # 입력 토큰 벡터의 크기
    self.dense_dim = dense_dim # 내부 밀집 층의 크기
    self.num_heads = num_heads # 어텐션 헤드의 크기
    self.attention = layers.MultiHeadAttention(
        num_heads = num_heads,
        key_dim = embed_dim,
    )
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation = "relu"),
         layers.Dense(embed_dim),]
    )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask = None): # call() 메서드에서 연산을 수행함
    if mask is not None:
      mask = mask[:, tf.newaxis, :]  # Embedding 층에서 생성하는 마스크는 2D이지만, 어텐션 층은 3D 또는 4D를 기대하므로
    attention_output = self.attention(inputs, inputs, attention_mask = mask) # 랭크를 늘림
    proj_input = self.layernorm_1(layers.add([attention_output, inputs]))
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(layers.add([proj_output, proj_input]))

  def get_config(self): # 모델을 저장할 수 있도록 직렬화를 구현
    config = super().get_config()
    config.update({
        "embed_dim" : self.embed_dim,
        "num_heads" : self.num_heads,
        "dense_dim" : self.dense_dim,
    })
    return config

In [6]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size = batch_size
)

text_only_train_ds = train_ds.map(lambda x, y : x)

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length, # 적당한 입력 크기를 유지하기 위해 입력에서 600개 단어 이후는 잘라 버림
    # 평균 리뷰 길이가 233개의 단어고 600개의 단어보다 긴 리뷰는 5%뿐이므로 합리적인 선택임.
)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_val_ds = val_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)

int_test_ds = test_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls = 4)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


이제 TransformerEncoder를 구현했으므로 이를 사용하여 LSTM 기반 모델과 비슷한 텍스트 분류 모델을 만들어 보자.

In [7]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape = (None,), dtype = "int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x) # TransformerEncoder가 전체 시퀀스를 반환하기 때문에
# 분류 작업을 위해 전역 풀링 층으로 각 시퀀스를 하나의 벡터로 만든다.
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer = "rmsprop",
    loss = "binary_crossentropy",
    metrics = ['accuracy']
)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 256)           │         543,776 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

모델을 훈련해보자  


In [8]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "transformer_encoder.keras",
        save_best_only = True)
]

model.fit(int_train_ds, validation_data = int_val_ds, epochs = 20,
          callbacks=callbacks)

model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects = {
        "TransformerEncoder" : TransformerEncoder
    }
)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 98ms/step - accuracy: 0.5392 - loss: 0.8714 - val_accuracy: 0.8292 - val_loss: 0.3904
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 72s 93ms/step - accuracy: 0.8128 - loss: 0.4104 - val_accuracy: 0.8462 - val_loss: 0.3562
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 92ms/step - accuracy: 0.8467 - loss: 0.3532 - val_accuracy: 0.8610 - val_loss: 0.3210
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 58s 93ms/step - accuracy: 0.8641 - loss: 0.3226 - val_accuracy: 0.8690 - val_loss: 0.3168
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 59s 94ms/step - accuracy: 0.8732 - loss: 0.3005 - val_accuracy: 0.8710 - val_loss: 0.3083
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 95ms/step - accuracy: 0.8849 - loss: 0.2737 - val_accuracy: 0.8632 - val_loss: 0.3320
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 59s 95ms/step - accuracy: 0.8951 - loss: 0.2538 - val_accuracy: 0.8696 - val_loss: 0.3047
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 95ms/step - accuracy: 0.8991 - loss: 0.2414 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [9]:
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.8716 - loss: 0.3078
테스트 정확도 : 0.871


## 위치 인코딩을 사용해서 위치 정보 주입하기

In [16]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)

        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)

        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]  # Get the batch size
        length = tf.shape(inputs)[-1]  # Get the sequence length

        # Create the positions tensor for the sequence length
        positions = tf.range(start=0, limit=length, delta=1)

        # Expand dimensions to match batch size
        positions = tf.expand_dims(positions, 0)  # Add a batch dimension
        positions = tf.tile(positions, [batch_size, 1])  # Repeat positions for each sample in the batch

        # Get embeddings for tokens and positions
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)

        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Return the mask for padding (0)
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


### 텍스트 분류 트랜스포머

In [17]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape = (None,), dtype = "int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x) # TransformerEncoder가 전체 시퀀스를 반환하기 때문에
# 분류 작업을 위해 전역 풀링 층으로 각 시퀀스를 하나의 벡터로 만든다.
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer = "rmsprop",
    loss = "binary_crossentropy",
    metrics = ['accuracy']
)

In [19]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, None, 256)           │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder                  │ (None, None, 256)           │         543,776 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,328,068 (43.21 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,664,035 (21.61 MB)

In [20]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "full_transformer_encoder.keras",
        save_best_only = True)
]

model.fit(int_train_ds, validation_data = int_val_ds, epochs = 20,
          callbacks=callbacks)

model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects = {
        "TransformerEncoder" : TransformerEncoder,
        "PositionalEmbedding" : PositionalEmbedding
    }
)


Epoch 1/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 97ms/step - accuracy: 0.9257 - loss: 0.1888 - val_accuracy: 0.8736 - val_loss: 0.3094
Epoch 2/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 97ms/step - accuracy: 0.9334 - loss: 0.1728 - val_accuracy: 0.8756 - val_loss: 0.3387
Epoch 3/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 95ms/step - accuracy: 0.9412 - loss: 0.1559 - val_accuracy: 0.8740 - val_loss: 0.3569
Epoch 4/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 95ms/step - accuracy: 0.9474 - loss: 0.1391 - val_accuracy: 0.8722 - val_loss: 0.3406
Epoch 5/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 93ms/step - accuracy: 0.9566 - loss: 0.1204 - val_accuracy: 0.8684 - val_loss: 0.3717
Epoch 6/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 57s 91ms/step - accuracy: 0.9617 - loss: 0.1069 - val_accuracy: 0.8710 - val_loss: 0.3962
Epoch 7/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 90ms/step - accuracy: 0.9652 - loss: 0.0969 - val_accuracy: 0.8668 - val_loss: 0.4086
Epoch 8/20
625/625 ━━━━━━━━━━━━━━━━━━━━ 55s 88ms/step - accuracy: 0.9729 - loss: 0.0777 - 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [21]:
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.8598 - loss: 0.3395
테스트 정확도 : 0.861
